# Import Packages and Files

In [1]:
import torch.nn as nn
from functionalities import dataloader as dl
from functionalities import evaluater as ev
from functionalities import filemanager as fm
from functionalities import trainer as tr
from architecture import RotNet as RN

# Load Dataset

In [2]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 0, 128)

# Initialize Loss Criterion

In [4]:
criterion = nn.CrossEntropyLoss()

# Train RotNet for Rotation Task and Classifiers on Feature Maps 

In [5]:
# set rot classes
rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']

## 3 Block RotNet

In [6]:
# initialize network
net_block3 = RN.RotNet(num_classes=4, num_conv_block=3, add_avg_pool=False)

In [7]:
# train network
rot_block3_loss_log, rot_block3_valid_accuracy_log, rot_block3_test_accuracy_log, rot_block3_max_accuracy, \
rot_block3_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net_block3, 
                                             criterion, trainloader, None, testloader, rot=['90', '180', '270'])

[1, 60] loss: 1.136
[1, 120] loss: 0.994
[1, 180] loss: 0.926
[1, 240] loss: 0.866
[1, 300] loss: 0.812
[1, 360] loss: 0.788
Epoch: 1 -> Loss: 0.622136056423
Epoch: 1 -> Test Accuracy: 70.145
[2, 60] loss: 0.737
[2, 120] loss: 0.700
[2, 180] loss: 0.694
[2, 240] loss: 0.666
[2, 300] loss: 0.648
[2, 360] loss: 0.627
Epoch: 2 -> Loss: 0.640503168106
Epoch: 2 -> Test Accuracy: 75.9675
[3, 60] loss: 0.618
[3, 120] loss: 0.608
[3, 180] loss: 0.589
[3, 240] loss: 0.578
[3, 300] loss: 0.569
[3, 360] loss: 0.552
Epoch: 3 -> Loss: 0.608837962151
Epoch: 3 -> Test Accuracy: 77.955
[4, 60] loss: 0.538
[4, 120] loss: 0.543
[4, 180] loss: 0.521
[4, 240] loss: 0.529
[4, 300] loss: 0.520
[4, 360] loss: 0.511
Epoch: 4 -> Loss: 0.438581287861
Epoch: 4 -> Test Accuracy: 79.2975
[5, 60] loss: 0.505
[5, 120] loss: 0.501
[5, 180] loss: 0.488
[5, 240] loss: 0.496
[5, 300] loss: 0.491
[5, 360] loss: 0.494
Epoch: 5 -> Loss: 0.529520392418
Epoch: 5 -> Test Accuracy: 81.4275
[6, 60] loss: 0.469
[6, 120] loss: 0.

[42, 240] loss: 0.309
[42, 300] loss: 0.298
[42, 360] loss: 0.321
Epoch: 42 -> Loss: 0.376379430294
Epoch: 42 -> Test Accuracy: 86.2675
[43, 60] loss: 0.293
[43, 120] loss: 0.311
[43, 180] loss: 0.289
[43, 240] loss: 0.314
[43, 300] loss: 0.318
[43, 360] loss: 0.310
Epoch: 43 -> Loss: 0.433914572001
Epoch: 43 -> Test Accuracy: 86.505
[44, 60] loss: 0.289
[44, 120] loss: 0.298
[44, 180] loss: 0.313
[44, 240] loss: 0.312
[44, 300] loss: 0.295
[44, 360] loss: 0.324
Epoch: 44 -> Loss: 0.233928084373
Epoch: 44 -> Test Accuracy: 86.46
[45, 60] loss: 0.298
[45, 120] loss: 0.296
[45, 180] loss: 0.316
[45, 240] loss: 0.301
[45, 300] loss: 0.314
[45, 360] loss: 0.314
Epoch: 45 -> Loss: 0.373682260513
Epoch: 45 -> Test Accuracy: 86.8825
[46, 60] loss: 0.304
[46, 120] loss: 0.312
[46, 180] loss: 0.295
[46, 240] loss: 0.307
[46, 300] loss: 0.307
[46, 360] loss: 0.312
Epoch: 46 -> Loss: 0.465635359287
Epoch: 46 -> Test Accuracy: 86.6075
[47, 60] loss: 0.297
[47, 120] loss: 0.309
[47, 180] loss: 0.30

[83, 240] loss: 0.153
[83, 300] loss: 0.155
[83, 360] loss: 0.162
Epoch: 83 -> Loss: 0.103295899928
Epoch: 83 -> Test Accuracy: 90.4825
[84, 60] loss: 0.133
[84, 120] loss: 0.141
[84, 180] loss: 0.153
[84, 240] loss: 0.148
[84, 300] loss: 0.147
[84, 360] loss: 0.151
Epoch: 84 -> Loss: 0.147079840302
Epoch: 84 -> Test Accuracy: 90.51
[85, 60] loss: 0.139
[85, 120] loss: 0.138
[85, 180] loss: 0.152
[85, 240] loss: 0.146
[85, 300] loss: 0.153
[85, 360] loss: 0.150
Epoch: 85 -> Loss: 0.214863091707
Epoch: 85 -> Test Accuracy: 90.4625
[86, 60] loss: 0.138
[86, 120] loss: 0.138
[86, 180] loss: 0.143
[86, 240] loss: 0.148
[86, 300] loss: 0.138
[86, 360] loss: 0.156
Epoch: 86 -> Loss: 0.222716003656
Epoch: 86 -> Test Accuracy: 90.4325
[87, 60] loss: 0.145
[87, 120] loss: 0.133
[87, 180] loss: 0.140
[87, 240] loss: 0.146
[87, 300] loss: 0.151
[87, 360] loss: 0.159
Epoch: 87 -> Loss: 0.0961948111653
Epoch: 87 -> Test Accuracy: 90.4175
[88, 60] loss: 0.133
[88, 120] loss: 0.125
[88, 180] loss: 0.

[123, 300] loss: 0.058
[123, 360] loss: 0.053
Epoch: 123 -> Loss: 0.0764970779419
Epoch: 123 -> Test Accuracy: 92.3325
[124, 60] loss: 0.050
[124, 120] loss: 0.050
[124, 180] loss: 0.049
[124, 240] loss: 0.052
[124, 300] loss: 0.049
[124, 360] loss: 0.050
Epoch: 124 -> Loss: 0.0354960076511
Epoch: 124 -> Test Accuracy: 92.375
[125, 60] loss: 0.043
[125, 120] loss: 0.043
[125, 180] loss: 0.047
[125, 240] loss: 0.045
[125, 300] loss: 0.049
[125, 360] loss: 0.049
Epoch: 125 -> Loss: 0.077633947134
Epoch: 125 -> Test Accuracy: 92.27
[126, 60] loss: 0.041
[126, 120] loss: 0.047
[126, 180] loss: 0.042
[126, 240] loss: 0.043
[126, 300] loss: 0.046
[126, 360] loss: 0.047
Epoch: 126 -> Loss: 0.0515734478831
Epoch: 126 -> Test Accuracy: 92.3
[127, 60] loss: 0.038
[127, 120] loss: 0.040
[127, 180] loss: 0.042
[127, 240] loss: 0.046
[127, 300] loss: 0.041
[127, 360] loss: 0.044
Epoch: 127 -> Loss: 0.0225582513958
Epoch: 127 -> Test Accuracy: 92.275
[128, 60] loss: 0.037
[128, 120] loss: 0.043
[128

Epoch: 162 -> Loss: 0.0166986938566
Epoch: 162 -> Test Accuracy: 92.1075
[163, 60] loss: 0.014
[163, 120] loss: 0.013
[163, 180] loss: 0.014
[163, 240] loss: 0.014
[163, 300] loss: 0.015
[163, 360] loss: 0.014
Epoch: 163 -> Loss: 0.0162970721722
Epoch: 163 -> Test Accuracy: 92.05
[164, 60] loss: 0.014
[164, 120] loss: 0.014
[164, 180] loss: 0.014
[164, 240] loss: 0.014
[164, 300] loss: 0.013
[164, 360] loss: 0.013
Epoch: 164 -> Loss: 0.00841785594821
Epoch: 164 -> Test Accuracy: 92.0275
[165, 60] loss: 0.013
[165, 120] loss: 0.013
[165, 180] loss: 0.012
[165, 240] loss: 0.013
[165, 300] loss: 0.013
[165, 360] loss: 0.013
Epoch: 165 -> Loss: 0.0120249781758
Epoch: 165 -> Test Accuracy: 92.0225
[166, 60] loss: 0.012
[166, 120] loss: 0.012
[166, 180] loss: 0.013
[166, 240] loss: 0.014
[166, 300] loss: 0.013
[166, 360] loss: 0.013
Epoch: 166 -> Loss: 0.00935164559633
Epoch: 166 -> Test Accuracy: 91.97
[167, 60] loss: 0.012
[167, 120] loss: 0.013
[167, 180] loss: 0.012
[167, 240] loss: 0.01

In [6]:
load_net_block3 = True

if load_net_block3:
    net_block3 = fm.load_net("RotNet_rotation_200")

In [8]:
# train NonLinearClassifiers on feature map of net_3block
block3_loss_log, block3_valid_accuracy_log, block3_test_accuracy_log, block3_max_accuracy, block3_best_epoch = \
tr.train_all_blocks(3, 10, [0.1, 0.02, 0.004, 0.0008], [20, 40, 45, 100], 0.9, 5e-4, net_block3, criterion, trainloader,
                    None, testloader) 

[1, 60] loss: 2.162
[1, 120] loss: 1.231
[1, 180] loss: 1.133
[1, 240] loss: 1.057
[1, 300] loss: 1.026
[1, 360] loss: 0.982
Epoch: 1 -> Loss: 0.990146994591
Epoch: 1 -> Test Accuracy: 68.47
[2, 60] loss: 0.915
[2, 120] loss: 0.919
[2, 180] loss: 0.883
[2, 240] loss: 0.892
[2, 300] loss: 0.867
[2, 360] loss: 0.856
Epoch: 2 -> Loss: 0.903752207756
Epoch: 2 -> Test Accuracy: 72.74
[3, 60] loss: 0.817
[3, 120] loss: 0.817
[3, 180] loss: 0.798
[3, 240] loss: 0.801
[3, 300] loss: 0.777
[3, 360] loss: 0.791
Epoch: 3 -> Loss: 0.756074309349
Epoch: 3 -> Test Accuracy: 74.23
[4, 60] loss: 0.747
[4, 120] loss: 0.754
[4, 180] loss: 0.741
[4, 240] loss: 0.724
[4, 300] loss: 0.756
[4, 360] loss: 0.727
Epoch: 4 -> Loss: 0.601166129112
Epoch: 4 -> Test Accuracy: 75.25
[5, 60] loss: 0.719
[5, 120] loss: 0.715
[5, 180] loss: 0.713
[5, 240] loss: 0.711
[5, 300] loss: 0.700
[5, 360] loss: 0.715
Epoch: 5 -> Loss: 0.765639603138
Epoch: 5 -> Test Accuracy: 76.35
[6, 60] loss: 0.679
[6, 120] loss: 0.675
[6, 

Epoch: 42 -> Loss: 0.173453181982
Epoch: 42 -> Test Accuracy: 83.14
[43, 60] loss: 0.312
[43, 120] loss: 0.310
[43, 180] loss: 0.291
[43, 240] loss: 0.292
[43, 300] loss: 0.300
[43, 360] loss: 0.308
Epoch: 43 -> Loss: 0.450003147125
Epoch: 43 -> Test Accuracy: 82.91
[44, 60] loss: 0.300
[44, 120] loss: 0.281
[44, 180] loss: 0.298
[44, 240] loss: 0.281
[44, 300] loss: 0.304
[44, 360] loss: 0.294
Epoch: 44 -> Loss: 0.404316902161
Epoch: 44 -> Test Accuracy: 83.25
[45, 60] loss: 0.282
[45, 120] loss: 0.300
[45, 180] loss: 0.287
[45, 240] loss: 0.289
[45, 300] loss: 0.288
[45, 360] loss: 0.288
Epoch: 45 -> Loss: 0.344729572535
Epoch: 45 -> Test Accuracy: 83.13
[46, 60] loss: 0.288
[46, 120] loss: 0.272
[46, 180] loss: 0.281
[46, 240] loss: 0.283
[46, 300] loss: 0.277
[46, 360] loss: 0.281
Epoch: 46 -> Loss: 0.297571182251
Epoch: 46 -> Test Accuracy: 83.18
[47, 60] loss: 0.289
[47, 120] loss: 0.276
[47, 180] loss: 0.270
[47, 240] loss: 0.262
[47, 300] loss: 0.272
[47, 360] loss: 0.267
Epoch

[84, 60] loss: 0.217
[84, 120] loss: 0.230
[84, 180] loss: 0.224
[84, 240] loss: 0.220
[84, 300] loss: 0.240
[84, 360] loss: 0.232
Epoch: 84 -> Loss: 0.252890765667
Epoch: 84 -> Test Accuracy: 83.23
[85, 60] loss: 0.224
[85, 120] loss: 0.232
[85, 180] loss: 0.230
[85, 240] loss: 0.233
[85, 300] loss: 0.220
[85, 360] loss: 0.227
Epoch: 85 -> Loss: 0.411238521338
Epoch: 85 -> Test Accuracy: 83.36
[86, 60] loss: 0.233
[86, 120] loss: 0.223
[86, 180] loss: 0.228
[86, 240] loss: 0.225
[86, 300] loss: 0.209
[86, 360] loss: 0.220
Epoch: 86 -> Loss: 0.225403144956
Epoch: 86 -> Test Accuracy: 83.32
[87, 60] loss: 0.222
[87, 120] loss: 0.220
[87, 180] loss: 0.226
[87, 240] loss: 0.231
[87, 300] loss: 0.228
[87, 360] loss: 0.211
Epoch: 87 -> Loss: 0.223494291306
Epoch: 87 -> Test Accuracy: 83.32
[88, 60] loss: 0.233
[88, 120] loss: 0.223
[88, 180] loss: 0.234
[88, 240] loss: 0.223
[88, 300] loss: 0.223
[88, 360] loss: 0.225
Epoch: 88 -> Loss: 0.289132535458
Epoch: 88 -> Test Accuracy: 83.28
[89, 

[25, 300] loss: 0.223
[25, 360] loss: 0.224
Epoch: 25 -> Loss: 0.322388321161
Epoch: 25 -> Test Accuracy: 85.65
[26, 60] loss: 0.214
[26, 120] loss: 0.225
[26, 180] loss: 0.212
[26, 240] loss: 0.231
[26, 300] loss: 0.225
[26, 360] loss: 0.222
Epoch: 26 -> Loss: 0.386886686087
Epoch: 26 -> Test Accuracy: 85.59
[27, 60] loss: 0.208
[27, 120] loss: 0.220
[27, 180] loss: 0.208
[27, 240] loss: 0.209
[27, 300] loss: 0.217
[27, 360] loss: 0.210
Epoch: 27 -> Loss: 0.34619101882
Epoch: 27 -> Test Accuracy: 85.71
[28, 60] loss: 0.198
[28, 120] loss: 0.215
[28, 180] loss: 0.216
[28, 240] loss: 0.208
[28, 300] loss: 0.214
[28, 360] loss: 0.204
Epoch: 28 -> Loss: 0.256702274084
Epoch: 28 -> Test Accuracy: 85.66
[29, 60] loss: 0.194
[29, 120] loss: 0.201
[29, 180] loss: 0.199
[29, 240] loss: 0.207
[29, 300] loss: 0.215
[29, 360] loss: 0.215
Epoch: 29 -> Loss: 0.249454781413
Epoch: 29 -> Test Accuracy: 85.64
[30, 60] loss: 0.189
[30, 120] loss: 0.199
[30, 180] loss: 0.202
[30, 240] loss: 0.216
[30, 3

Epoch: 66 -> Loss: 0.0875278636813
Epoch: 66 -> Test Accuracy: 86.62
[67, 60] loss: 0.090
[67, 120] loss: 0.093
[67, 180] loss: 0.097
[67, 240] loss: 0.098
[67, 300] loss: 0.095
[67, 360] loss: 0.091
Epoch: 67 -> Loss: 0.279062747955
Epoch: 67 -> Test Accuracy: 86.56
[68, 60] loss: 0.097
[68, 120] loss: 0.097
[68, 180] loss: 0.091
[68, 240] loss: 0.100
[68, 300] loss: 0.092
[68, 360] loss: 0.095
Epoch: 68 -> Loss: 0.0385071113706
Epoch: 68 -> Test Accuracy: 86.58
[69, 60] loss: 0.096
[69, 120] loss: 0.086
[69, 180] loss: 0.098
[69, 240] loss: 0.095
[69, 300] loss: 0.095
[69, 360] loss: 0.100
Epoch: 69 -> Loss: 0.0830150395632
Epoch: 69 -> Test Accuracy: 86.55
[70, 60] loss: 0.092
[70, 120] loss: 0.095
[70, 180] loss: 0.092
[70, 240] loss: 0.100
[70, 300] loss: 0.098
[70, 360] loss: 0.091
Epoch: 70 -> Loss: 0.147570535541
Epoch: 70 -> Test Accuracy: 86.54
[71, 60] loss: 0.092
[71, 120] loss: 0.095
[71, 180] loss: 0.094
[71, 240] loss: 0.097
[71, 300] loss: 0.087
[71, 360] loss: 0.095
Ep

[8, 60] loss: 1.487
[8, 120] loss: 1.479
[8, 180] loss: 1.488
[8, 240] loss: 1.475
[8, 300] loss: 1.487
[8, 360] loss: 1.480
Epoch: 8 -> Loss: 1.39841389656
Epoch: 8 -> Test Accuracy: 44.18
[9, 60] loss: 1.474
[9, 120] loss: 1.471
[9, 180] loss: 1.499
[9, 240] loss: 1.486
[9, 300] loss: 1.474
[9, 360] loss: 1.473
Epoch: 9 -> Loss: 1.40985012054
Epoch: 9 -> Test Accuracy: 44.87
[10, 60] loss: 1.468
[10, 120] loss: 1.463
[10, 180] loss: 1.460
[10, 240] loss: 1.483
[10, 300] loss: 1.447
[10, 360] loss: 1.469
Epoch: 10 -> Loss: 1.56842362881
Epoch: 10 -> Test Accuracy: 43.93
[11, 60] loss: 1.469
[11, 120] loss: 1.460
[11, 180] loss: 1.476
[11, 240] loss: 1.464
[11, 300] loss: 1.467
[11, 360] loss: 1.446
Epoch: 11 -> Loss: 1.55275261402
Epoch: 11 -> Test Accuracy: 44.59
[12, 60] loss: 1.471
[12, 120] loss: 1.435
[12, 180] loss: 1.458
[12, 240] loss: 1.470
[12, 300] loss: 1.470
[12, 360] loss: 1.465
Epoch: 12 -> Loss: 1.53223562241
Epoch: 12 -> Test Accuracy: 44.68
[13, 60] loss: 1.473
[13, 

[49, 360] loss: 1.176
Epoch: 49 -> Loss: 1.12264847755
Epoch: 49 -> Test Accuracy: 53.43
[50, 60] loss: 1.160
[50, 120] loss: 1.153
[50, 180] loss: 1.169
[50, 240] loss: 1.190
[50, 300] loss: 1.155
[50, 360] loss: 1.153
Epoch: 50 -> Loss: 1.2540705204
Epoch: 50 -> Test Accuracy: 53.44
[51, 60] loss: 1.178
[51, 120] loss: 1.159
[51, 180] loss: 1.162
[51, 240] loss: 1.160
[51, 300] loss: 1.167
[51, 360] loss: 1.167
Epoch: 51 -> Loss: 1.0436899662
Epoch: 51 -> Test Accuracy: 54.03
[52, 60] loss: 1.161
[52, 120] loss: 1.167
[52, 180] loss: 1.169
[52, 240] loss: 1.169
[52, 300] loss: 1.182
[52, 360] loss: 1.158
Epoch: 52 -> Loss: 1.2312848568
Epoch: 52 -> Test Accuracy: 53.62
[53, 60] loss: 1.157
[53, 120] loss: 1.176
[53, 180] loss: 1.160
[53, 240] loss: 1.145
[53, 300] loss: 1.174
[53, 360] loss: 1.146
Epoch: 53 -> Loss: 0.86311852932
Epoch: 53 -> Test Accuracy: 53.58
[54, 60] loss: 1.151
[54, 120] loss: 1.180
[54, 180] loss: 1.166
[54, 240] loss: 1.151
[54, 300] loss: 1.160
[54, 360] los

[91, 60] loss: 1.156
[91, 120] loss: 1.133
[91, 180] loss: 1.121
[91, 240] loss: 1.136
[91, 300] loss: 1.130
[91, 360] loss: 1.132
Epoch: 91 -> Loss: 1.09939694405
Epoch: 91 -> Test Accuracy: 54.37
[92, 60] loss: 1.145
[92, 120] loss: 1.118
[92, 180] loss: 1.112
[92, 240] loss: 1.145
[92, 300] loss: 1.108
[92, 360] loss: 1.145
Epoch: 92 -> Loss: 1.24633836746
Epoch: 92 -> Test Accuracy: 54.29
[93, 60] loss: 1.136
[93, 120] loss: 1.124
[93, 180] loss: 1.138
[93, 240] loss: 1.124
[93, 300] loss: 1.132
[93, 360] loss: 1.119
Epoch: 93 -> Loss: 1.10955047607
Epoch: 93 -> Test Accuracy: 54.27
[94, 60] loss: 1.124
[94, 120] loss: 1.106
[94, 180] loss: 1.142
[94, 240] loss: 1.140
[94, 300] loss: 1.127
[94, 360] loss: 1.129
Epoch: 94 -> Loss: 1.20155501366
Epoch: 94 -> Test Accuracy: 54.45
[95, 60] loss: 1.118
[95, 120] loss: 1.138
[95, 180] loss: 1.129
[95, 240] loss: 1.133
[95, 300] loss: 1.136
[95, 360] loss: 1.130
Epoch: 95 -> Loss: 0.918730735779
Epoch: 95 -> Test Accuracy: 54.25
[96, 60] 

In [7]:
# train ConvClassifiers on feature map of net_3block
conv_block3_loss_log, conv_block3_valid_accuracy_log, conv_block3_test_accuracy_log, conv_block3_max_accuracy, \
conv_block3_best_epoch = tr.train_all_blocks(3, 10, [0.1, 0.02, 0.004, 0.0008], [35, 70, 85, 100], 0.9, 5e-4, net_block3, 
                                            criterion, trainloader, None, testloader, use_ConvClassifier=True) 

RuntimeError: Given groups=1, weight[192, 24576, 3, 3], so expected input[128, 96, 16, 16] to have 24576 channels, but got 96 channels instead

In [ ]:
# rename files
fm.add_block_to_name(3, [100, 200], list(rot_block3_best_epoch) + block3_best_epoch + conv_block3_best_epoch)

## 4 Block RotNet

In [ ]:
# initialize network
net_block4 = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

In [ ]:
# train network
rot_block4_loss_log, rot_block4_valid_accuracy_log, rot_block4_test_accuracy_log, rot_block4_max_accuracy, \
rot_block4_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net_block4, 
                                             criterion, trainloader, None, testloader, rot=['90', '180', '270'])

In [ ]:
# train NonLinearClassifiers on feature map of net_3block
block4_loss_log, block4_valid_accuracy_log, block4_test_accuracy_log, block4_max_accuracy, block4_best_epoch = \
tr.train_all_blocks(4, 10, [0.1, 0.02, 0.004, 0.0008], [20, 40, 45, 100], 0.9, 5e-4, net_block4, criterion, trainloader,
                    None, testloader) 

In [ ]:
# train ConvClassifiers on feature map of net_3block
conv_block4_loss_log, conv_block4_valid_accuracy_log, conv_block4_test_accuracy_log, conv_block4_max_accuracy, \
conv_block4_best_epoch = tr.train_all_blocks(4, 10, [0.1, 0.02, 0.004, 0.0008], [35, 70, 85, 100], 0.9, 5e-4, net_block4, 
                                            criterion, trainloader, None, testloader, use_ConvClassifier=True) 

In [ ]:
# rename files
fm.add_block_to_name(4, [100, 200], list(rot_block4_best_epoch) + block4_best_epoch + conv_block4_best_epoch)

## 5 Block RotNet

In [ ]:
# initialize network
net_block5 = RN.RotNet(num_classes=4, num_conv_block=5, add_avg_pool=False)

In [ ]:
# train network
rot_block5_loss_log, rot_block5_valid_accuracy_log, rot_block5_test_accuracy_log, rot_block5_max_accuracy, \
rot_block5_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net_block5, 
                                             criterion, trainloader, None, testloader, rot=['90', '180', '270'])

In [ ]:
# train NonLinearClassifiers on feature map of net_3block
block5_loss_log, block5_valid_accuracy_log, block5_test_accuracy_log, block5_max_accuracy, block5_best_epoch = \
tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], [20, 40, 45, 100], 0.9, 5e-4, net_block5, criterion, trainloader,
                    None, testloader) 

In [ ]:
# train ConvClassifiers on feature map of net_3block
conv_block5_loss_log, conv_block5_valid_accuracy_log, conv_block5_test_accuracy_log, conv_block5_max_accuracy, \
conv_block5_best_epoch = tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], [35, 70, 85, 100], 0.9, 5e-4, net_block5, 
                                            criterion, trainloader, None, testloader, use_ConvClassifier=True) 

In [ ]:
# rename files
fm.add_block_to_name(5, [100, 200], list(rot_block5_best_epoch) + block5_best_epoch + conv_block5_best_epoch)

## 5 Block RotNet with Average Pooling after ConvBlock 3

In [ ]:
# initialize network
net_block5_avg = RN.RotNet(num_classes=4, num_conv_block=5, add_avg_pool=True)

In [ ]:
# train network
rot_block5_avg_loss_log, rot_block5_avg_valid_accuracy_log, rot_block5_avg_test_accuracy_log, \
rot_block5_avg_max_accuracy, rot_block5_avg_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net_block5_avg, criterion, trainloader, None, testloader, rot=['90', '180', '270'])

In [ ]:
# train NonLinearClassifiers on feature map of net_3block
block5_avg_loss_log, block5_avg_valid_accuracy_log, block5_avg_test_accuracy_log, block5_avg_max_accuracy, \
block5_best_epoch = tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], [20, 40, 45, 100], 0.9, 5e-4, net_block5_avg, 
                                        criterion, trainloader, None, testloader) 

In [ ]:
# train ConvClassifiers on feature map of net_3block
conv_block5_avg_loss_log, conv_block5_avg_valid_accuracy_log, conv_block5_avg_test_accuracy_log, \
conv_block5_avg_max_accuracy, conv_block5_avg_best_epoch = tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], \
    [35, 70, 85, 100], 0.9, 5e-4, net_block5_avg, criterion, trainloader, None, testloader, use_ConvClassifier=True) 

In [ ]:
# rename files
fm.add_block_to_name(5, [100, 200], list(rot_block5_avg_best_epoch) + block5_avg_best_epoch + conv_block5_avg_best_epoch)

# Supervised NIN

Note: In the code of the paper a 3 convolutional block RotNet was used for the classification task.

In [ ]:
# initialize networks
net_class = RN.RotNet(num_classes=10, num_conv_block=3, add_avg_pool=False)

In [ ]:
# train 3 block RotNet on classification task
class_NIN_loss_log, class_NIN_valid_accuracy_log, class_NIN_test_accuracy_log, class_NIN_max_accuracy, \
class_NIN_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net_class, 
                                            criterion, trainloader, None, testloader)

# Test Accuracy